#Installs


usar um arquivo requeriments.txt -- coloca os modulos e versoes -- pip install -r requiriments.txt



In [1]:
!pip install anvil-uplink

     |████████████████████████████████| 62 kB 1.1 MB/s 
     |████████████████████████████████| 51 kB 179 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45230 sha256=ee85f8cebf65c0c4077fd50a14151ec5e187bfc45a5e6daaea4114af240e6d7a
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py


In [2]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 4.7 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619441 sha256=2f850ce65769fcc6cefbd80906db2a9f14a1842eeacceee62ec175816a82dce4
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [3]:
!pip install requests

#Imports

In [4]:
# Anvil
import anvil.media
import anvil.tables as tables
import anvil.tables.query as q
from anvil.tables import app_tables
import anvil.server

# thread para a API
from threading import Thread
# para acionar a API no servidor do Anvil
import requests


# para gerar o id dos modelos
from uuid import uuid4

# surprise e toda a estrutura para IA
import sys
import os
import pickle
import time
from collections import defaultdict
from surprise.prediction_algorithms import knns
from surprise import SVD
from surprise import Dataset,  Trainset, Reader
import pandas as pd


# Esqueleto API/Colab/Frontend

In [5]:

# para conectar com a minha versao original do suggeri
#anvil.server.connect("BFSI2NTMXOF7JIPPVF5FMCBY-55DIUIXG7VVL3GVJ")

## para conectar com a versão do front-end do Geziel
##anvil.server.connect("KISG3R44OEKOOUBHCGY7WRUM-CD7JWV2AUK4WZKRN")

# para conectar a versao final da Erica 
#anvil.server.connect("GOV2WLOJBBXX5MY4ZP5PTBXD-QEV7AO4O3G54JYOU")
# para conectar a versao clone 5
anvil.server.connect("GZAXUQF7PPVME4AW7FIKSIP6-N3KNIIBKXBE7MBCM")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


#Front-end e API

In [6]:
## FRONTEND ##

status_modelo_cliente_1 = None

def frontend_solicita_treinamento():
  # arquivo que o cliente salvou no disco do frontend
  csv_filename = 'empresa1_csv.csv'
  # cria no banco o modelo para o cliente com o status "Recém Criado"
  global status_modelo_cliente_1
  status_modelo_cliente_1 = 'Recém Criado'
  # colab informa como sera o nome do modelo a ser criado no treinamento
  model_name = colab_servercall_treinar_modelo(csv_filename)
  return model_name

In [7]:
## API ##

def api_download_csv(csv_filename):
  csv_content = 'conteudo do arquivo csv do cliente 1'
  return csv_content

def api_treinamento_iniciado(model_name):
  # salva no banco que este modelo esta com o treinamento iniciado
  # busca no banco o modelo, atualiza para "Treinamento Iniciado"
  print(f'salvando modelo {model_name} como iniciado no banco de dados')
  global status_modelo_cliente_1
  status_modelo_cliente_1 = "Treinamento Iniciado"


def api_treinamento_finalizado(model_name):
  # salva no banco que este modelo esta treinado
  # busca no banco o modelo, atualiza para "Treinado"
  print(f'salvando modelo {model_name} como treinado no banco de dados')
  global status_modelo_cliente_1
  status_modelo_cliente_1 = "Treinado"


def api_predict_item(model_name, user_id, n=5):
  # chama o colab via servercall para realizar a predicao
  items = colab_predict_item(model_name, user_id, n)
  return items

#API - Chamadas do Anvil para o COLAB

In [8]:
## COLAB ##

def colab_servercall_treinar_modelo(csv_filename):
  # gera o nome do modelo (apenas 8 caracteres para ficar melhor)
  #model_name = str(uuid4())[:8]
  model_name = csv_filename[:8]
  # cria a thread para inciar treinamento
  t = Thread(target=colab_thread_treinar, args=(csv_filename, model_name, ))
  t.start()
  # retorna o nome modelo
  return model_name

def colab_predict_item(model_name, user_id, n=5):
  ## carrega modelo
  model_filename = f'{model_name}.model'
  print(f'modelo {model_name} carregado do arquivo {model_filename}')
  model = load_model(model_filename)  
  # model = carregar()
  # items = predict(model, user_id, n=5)
  itens = []
  itens = predict_user(model, user_id, n)
  return itens

def colab_predict_user_item(model_name, user_id, item_id):
  ## carrega modelo
  model_filename = f'{model_name}.model'
  print(f'modelo {model_name} carregado do arquivo {model_filename}')
  model = load_model(model_filename)
  # model = carregar()
  rating = predict_user_item(model, user_id=user_id, item_id=item_id)
  return rating 

def colab_predict_users(model_name, item_id, n=5):
  ## carrega modelo
  model_filename = f'{model_name}.model'
  print(f'modelo {model_name} carregado do arquivo {model_filename}')
  model = load_model(model_filename)  
    # items = predict(model, user_id, n=5)
  itens = []
  itens = predict_item(model, item_id, n)
  return itens   

In [9]:

@anvil.server.callable
def thread_treinar(csv_filename, model_name):
  return colab_thread_treinar(csv_filename, model_name)

@anvil.server.callable
def treinar_modelo(csv_filename):
  return  colab_servercall_treinar_modelo(csv_filename)

@anvil.server.callable
def predict_item(model_name, user_id, n=5):
  return colab_predict_item(model_name, user_id, n)

@anvil.server.callable
def predict_users(model_name, item_id, n=5):  
  return colab_predict_users(model_name, item_id, n=5)  

@anvil.server.callable
def predict_user_item(model_name, user_id, item_id):
  return colab_predict_user_item(model_name, user_id, item_id)

@anvil.server.callable
def get_uuid():
    return str(uuid4())[:8]

@anvil.server.callable
def colab_online():
  return True



# VERSÃO FINAL COM SURPRISE (Toda a parte de IA está aqui)

In [10]:

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        if n:
          top_n[uid] = user_ratings[:n]
        else:
          top_n[uid] = user_ratings

    return top_n

def train_model(csv_filename, n=None):
  start = time.time()
  # TODO: alterar para ler .csv
  print('loading from dataset')
  df = pd.read_csv(csv_filename)
  data = Dataset.load_from_df(df[['UserId', 'ProductId', 'Rating']], Reader(rating_scale=(1,5)))
  #data = Dataset.load_builtin('ml-100k')
  trainset = data.build_full_trainset()
  algo = SVD(random_state=1)
  algo.fit(trainset)

  # Than predict ratings for all pairs (u, i) that are NOT in the training set.
  testset = trainset.build_anti_testset()
  predictions = algo.test(testset)

  model = get_top_n(predictions, n=n)
  elapsed_seconds = time.time() - start
  print(f'model trained in {elapsed_seconds:.1f} seconds')
  return model

def save_model(model, filename=''):
  tamanho_modelo = sys.getsizeof(model)
  print(f'tamanho do modelo: {tamanho_modelo} bytes')
  if filename == '':
    file_name2 = str(uuid.uuid4())[:8] + '.model'
  else:
    file_name2 = filename + '.model'
  pickle.dump(model, open(file_name2, 'wb'))
  tamanho_arquivo = os.stat(file_name2).st_size
  print(f'tamanho do arquivo: {tamanho_arquivo} bytes')
  return file_name2

def load_model(file_name):
  print('Carregando o modelo')
  loaded_model = pickle.load(open(file_name, 'rb'))
  print ("Modelo carregado")
  return loaded_model

def predict_user(model, user_id, n=5):
  print('Predict Itens for a User')
  result = []
  # Print the recommended items for each user
  for uid, user_ratings in model.items():
    if str(uid) == str(user_id):
      result = [iid for (iid, _) in user_ratings][:n]
  print(result)
  return result

def predict_item(model, item_id, n=5):
  print('Predict Users for a Item')
  result = []
  # Print the recommended items for each user
  top_n_iid = defaultdict(list)
  for uid, user_ratings in model.items():
    for (iid, rating) in user_ratings:
      if str(iid) == str(item_id):
        top_n_iid[iid].append((uid, rating))        

  # Then sort the predictions for each user and retrieve the k highest ones.
  for iid, user_ratings in top_n_iid.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
  result = [uid for (uid, _) in user_ratings][:n]
  return result  

def predict_user_item(model, user_id, item_id):
  print('Predict Rating for a Item by User')
  result = []
  # Print the recommended items for each user
  for uid, user_ratings in model.items():
    if str(uid) == str(user_id):
      for (iid, rating) in user_ratings:
        if str(iid) == str(item_id):
          result.append(rating)
  return result





In [11]:
# esta função é acionada pelo anvil para que o colab/python receba o arquivo
# com os dados (csv), e inicia automaticamente o treinamento.


@anvil.server.callable
def get_data(file, modelo_id):
    print(modelo_id)
    with anvil.media.TempFile(file) as file_name:
        nome = modelo_id + '.csv'
        # file = open(nome,'w+')
        df = pd.read_csv(file_name)
        df.to_csv(nome)
        if(df.empty):
            return 'Não recebemos o arquivo, gentileza enviar novamente!'
        else:
            # training_model()
            colab_treinar_modelo(nome)
            print(df)
            return 'Arquivo Em Treinamento!'

In [12]:
@anvil.server.callable
def send_file(file):
    # print(modelo_id)
    with anvil.media.TempFile(file) as file_name:
        df = pd.read_csv(file_name)
        if(df.empty):
            return 'Não recebemos o arquivo, gentileza enviar novamente!'
        else:
            return colab_servercall_treinar_modelo(csv_filename) #retorna id modelo e treina numa thread

In [13]:
# exemplo
# /notify/start/:model_id e /notify/finish/:model_id

def notify_start(modelo_id):
  # executa o request para a api de acordo com os parametros
  site = "https://N3KNIIBKXBE7MBCM.anvil.app/3VIQN3574H2X3G3PBCYVOQBH/_/api/api_treinamento_iniciado/" + modelo_id
  #response = requests.get(f'https://cd7jwv2auk4wzkrn.anvil.app/_/private_api/NAUJWAV2RPBGYXSGAHBTFQXI/api_treinamento_iniciado/{modelo_id}')
  response = requests.get(site)
  # obtem o json de resposta 
  data = response.json()
  return data

def notify_finish(modelo_id):
  # executa o request para a api de acordo com os parametros
  site = "https://N3KNIIBKXBE7MBCM.anvil.app/3VIQN3574H2X3G3PBCYVOQBH/_/api/api_treinamento_finalizado/" + modelo_id
  response = requests.get(site)
  # response = requests.get(f'https://cd7jwv2auk4wzkrn.anvil.app/_/private_api/NAUJWAV2RPBGYXSGAHBTFQXI/api_treinamento_finalizado/{modelo_id}')
  # obtem o json de resposta
  data = response.json()
  return data

In [14]:
# esta é a função que efetivamente aciona o treinamento do modelo.
def colab_treinar_frontend(csv_filename):
  print('training model...')
  model_name = csv_filename[:8]
  
  # aciona o servidor anvil para informar que o modelo está treinado
  print('Aciona servidor Anvil para informar treinamento inicializado')
  print(notify_start(model_name))
  # site = "https://CD7JWV2AUK4WZKRN.anvil.app/_/private_api/NAUJWAV2RPBGYXSGAHBTFQXI/api_treinamento_iniciado/" + model_name
  # r = requests.get(site)
  # print(r.status_code)
  # print(r.headers)
  # print(r.content)

  model = train_model(csv_filename)
  print('saving model...')
  model_filename = save_model(model, model_name)
  print(f'model saved to {model_filename}')
  
  # aciona o servidor anvil para informar que o modelo está treinado
  print('Aciona servidor Anvil para informar treinamento finalizado')
  print(notify_finish(model_name))
  # site = "https://CD7JWV2AUK4WZKRN.anvil.app/_/private_api/NAUJWAV2RPBGYXSGAHBTFQXI/api_treinamento_finalizado/" + model_name
  # r = requests.get(site)
  # print(r.status_code)
  # print(r.headers)
  # print(r.content)

In [15]:
# esta funcao inicia o treinamento de um arquivo salvo no diretorio

def colab_treinar_modelo(csv_filename):
  model_name = csv_filename[:8]
  # cria a thread para inciar treinamento
  t = Thread(target=colab_treinar_frontend, args=(csv_filename, ))
  t.start()
  # retorna o nome modelo
  return model_name

# Inicia o servidor anvil para rodar até o fim dos tempos...

In [ ]:
################### aqui roda o serviço e faz todos os prints das APIs acionadas ####### essa é a melhor opção para rodas
anvil.server.wait_forever()